#  extract ALL Fund Transfer transactions of the accounts in the attached file

Hi @Gaurav and Team,

 

 

Good morning.

 

Can you please help us extract ALL Fund Transfer transactions of the accounts in the attached file from Oct. 01, 2023 to Oct. 31, 2023?

 

We are reviewing our current AML High Velocity rules in JIRA and we would need the ff. data. Thank you!

 

TSA
Transaction Amount
Transaction Created Date
Transaction Type (debit/credit)
 

 

Regards,

Mela

## Import libraries

In [1]:
# Import Libraries

import pandas as pd 
import numpy as np 

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

In [3]:
dfread = pd.read_excel(r"D:\MyStuff\Mela Sombillo csombillo1@tonikbank.com\all TSA and loan clients with topupscashinsusingDebitCard\Anti-Fraud Ops Request_18DEC23.xlsx", sheet_name='DS Request (Oct. 2023 Data)')
print(f"Data downloaded to dfread. The rows and columns in data are:\t {dfread.shape}")

Data downloaded to dfread. The rows and columns in data are:	 (787, 5)


In [4]:
dfread.columns

Index(['PID', 'TSA', 'Transaction Amount', 'Transaction Created Date',
       'Transaction Type (debit/credit)'],
      dtype='object')

In [14]:
dfread.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787 entries, 0 to 786
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   PID                              787 non-null    int64  
 1   TSA                              0 non-null      float64
 2   Transaction Amount               0 non-null      float64
 3   Transaction Created Date         0 non-null      float64
 4   Transaction Type (debit/credit)  0 non-null      float64
dtypes: float64(4), int64(1)
memory usage: 30.9 KB


In [16]:
dfread['PID'] = dfread['PID'].astype(str)

In [17]:
a = tuple(dfread['PID'])
print(a)

('1300639', '1111203', '1109895', '1086632', '1011901', '1103320', '1008748', '1088469', '1010092', '1635105', '2128061', '1069334', '1039739', '1001170', '1005197', '1857164', '1067929', '1033042', '1172353', '2236878', '1034626', '1493620', '1423446', '1012078', '2220521', '1020526', '1205846', '1020482', '1004297', '1217826', '1086586', '1016727', '1006465', '1004436', '1269149', '1605864', '1056644', '1017684', '1082113', '1080762', '1374499', '1002167', '1213794', '1419682', '1070582', '1591328', '1025770', '1084018', '1014300', '2163367', '1211513', '1047902', '1052329', '2096198', '1772256', '1623374', '1005620', '1013292', '1012769', '1016136', '1063095', '1074695', '1285794', '1192843', '1111093', '1091012', '1090732', '1041154', '1197989', '2022146', '1566620', '1154335', '1205590', '1017955', '1002221', '1075106', '1008245', '1000140', '1345746', '1018960', '1055454', '2066026', '1425143', '1085336', '1045819', '2046262', '1193190', '1016371', '1033793', '1082875', '1075155'

In [18]:
# Query

sq = f"""
select cust_id
, tsaaccountnumber
, to_account_no
, txn_ts
, status
, trx_amount
, trx_fee_amount 
, total_trx_amount
, trx_type
, transfervia
, transfer_description
, error_details
, merchant_name
, channel
, subchannel
 from prj-prod-dataplatform.risk_credit_mis.channel_transactions
 where date(txn_ts) between '2023-10-01' and '2023-10-31'
 and cust_id in {a}
 ;
 """
 
 

In [19]:
# Run the query to extract Data

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in DataFrame df is:\t{df.shape}")





Job ID de367fe7-3ac8-4f84-8a32-3a6801ee4adc successfully executed: 100%|██████████|



Downloading: 100%|██████████|
Query is running:   0%|          |
Query is running:   0%|          |
The rows and columns in DataFrame df is:	(12637, 15)


In [21]:
df.sort_values(by='txn_ts').sample(10)

,cust_id,tsaaccountnumber,to_account_no,txn_ts,status,trx_amount,trx_fee_amount,total_trx_amount,trx_type,transfervia,transfer_description,error_details,merchant_name,channel,subchannel
11544,2098950,60820989500002,60820989500002,2023-10-22 14:38:36,Success,860.0,NaN,860.0,CREDIT,instapay,None,None,None,InstaPay,G-Xchange/GCash
6067,1098539,60810985390008,60810985390008,2023-10-31 19:06:56,Success,2700.0,NaN,2700.0,CREDIT,instapay,None,None,None,InstaPay,Seabank
11717,1407529,60814075290001,09318610744,2023-10-22 17:59:27,Success,300.0,0.0,300.0,DEBIT,instapay,None,000001,None,InstaPay,G-Xchange/GCash
8906,2005254,60820052540008,60820052540008,2023-10-29 12:06:28,Success,2985.0,NaN,2985.0,CREDIT,instapay,None,None,None,InstaPay,G-Xchange/GCash
5473,2046262,60820462620002,09397282301,2023-10-28 17:17:40,Success,380.0,0.0,380.0,DEBIT,instapay,None,000001,None,InstaPay,G-Xchange/GCash
7045,2257856,60822578560003,09237439391,2023-10-30 09:14:48,Success,420.0,0.0,420.0,DEBIT,instapay,None,000001,None,InstaPay,G-Xchange/GCash
8729,2281883,60822818830007,09474769629,2023-10-29 13:55:13,Success,500.0,0.0,500.0,DEBIT,instapay,None,000001,None,InstaPay,G-Xchange/GCash
3266,2148368,60821483680005,047010013232,2023-10-27 23:29:41,Success,418.0,0.0,418.0,DEBIT,instapay,None,000001,None,InstaPay,Asia United Bank Corporation
7771,2128061,60821280610009,09106741220,2023-10-23 20:30:13,Success,21.0,0.0,21.0,DEBIT,instapay,None,000001,None,InstaPay,G-Xchange/GCash
9698,1111705,60811117050002,60811117050002,2023-10-13 08:07:58,Success,50000.0,NaN,50000.0,CREDIT,instapay,None,None,None,InstaPay,Philippine National Bank (PNB)


In [23]:
df.sort_values(by=['cust_id','txn_ts']).to_excel("AllFundtransfersfromoct1to31st.xlsx", sheet_name='Data', index=False)

In [24]:
dfread = dfread.merge(df, left_on='PID', right_on='cust_id', how = 'left')

In [25]:
dfread.sort_values(by=['PID','cust_id','txn_ts']).to_excel("AllFundtransfersfromoct1to31st.xlsx", sheet_name='Data', index=False)